In [162]:
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from toshl_fixer.data.expenses import get_expenses
import mlflow

In [163]:
expenses = get_expenses()
expenses

,id,desc,amount,category,date,tag
0,211141213,הדסה ירושלים,-2233.00,Health & Personal Care,2020-02-23,doctor
1,211141215,"בי""ח הדסה אם ו",-12.00,Leisure,2020-02-23,eating out
2,211141210,תן ביס,-52.00,Leisure,2020-02-23,eating out
3,211141212,פנגו-חניונים,-49.00,Transport,2020-02-23,parking
4,211141211,היר YELLOW פז,-254.07,Transport,2020-02-23,gaz
...,...,...,...,...,...,...
637,204285585,רולדין יהודה המכבי 🏷,-21.50,Basics,2019-10-01,groceries
638,204285587,שופרסל אקספרס יהודה מכבי 🏷,-13.70,Basics,2019-10-01,groceries
639,204330926,הפניקס אוטו חובה 🏷,-703.00,Taxes & Insurance,2019-10-01,insurance
640,204330925,PAYPAL *NETFLIX COM 35314369001 🏷,-46.90,Leisure,2019-10-01,missing


In [164]:
expenses['label'] = expenses['category'] + '/' + expenses['tag']
expenses['desc'] = expenses['desc'].str.strip('🏷')
expenses = expenses[expenses['amount'] < 0]

data_set = expenses[['desc', 'label']]

In [165]:
train, test = train_test_split(data_set, test_size=0.2)

In [166]:
def format_for_nltk(row):
    return {word: True for word in row["desc"].upper().split()}


train_x = train.apply(format_for_nltk, axis=1)
test_x = test.apply(format_for_nltk, axis=1)
train_y = train["label"]
test_y = test["label"]

In [167]:
train_set = list(zip(train_x, train_y))
test_set = list(zip(test_x, test_y))
train_set

[({'אופניים': True}, 'Transport/bicycle'),
 ({'ועד': True,
   'הבית': True,
   'וייצמן': True,
   '86': True,
   'תל': True,
   'אבי': True},
  'Home & Utilities/building upkeep'),
 ({'המכולת': True}, 'Leisure/eating out'),
 ({'טיב': True, 'טעם': True, 'רשתות': True}, 'Basics/groceries'),
 ({'SHI-SHI': True}, 'Leisure/eating out'),
 ({'טיב': True, 'טעם': True, 'יהודה': True, 'המכבי': True, 'ת"א': True},
  'Basics/groceries'),
 ({'כספומט': True, 'י': True, '832': True}, 'Home & Utilities/rent'),
 ({'לחם': True, 'תושיה': True, '-': True, 'יהודה': True, 'מכבי': True},
  'Basics/groceries'),
 ({'טיב': True, 'טעם': True, 'יהודה': True, 'המכבי': True, 'ת"א': True},
  'Basics/groceries'),
 ({'PAYPAL': True, '*NETFLIX': True, 'COM': True}, 'Leisure/apps'),
 ({'קיוסק': True, 'יהודה': True, 'מכ': True}, 'Basics/groceries'),
 ({'WIND': True, 'MOBILITY': True, 'GMBH': True, 'BERLIN': True, 'DE': True},
  'Transport/scooters'),
 ({'קפה': True, 'יסוד': True, 'המעלה': True}, 'Leisure/eating out'),
 (

In [168]:
with mlflow.start_run():
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    accuracy = nltk.classify.accuracy(classifier, test_set)

    print(f'{accuracy=}')
    mlflow.log_metric("accuracy", accuracy)
    
    train.to_csv('train-set.csv', index=False)
    mlflow.log_artifact('train-set.csv')
    
    test.to_csv('test-set.csv', index=False)
    mlflow.log_artifact('test-set.csv')
    
    
    

accuracy=0.5483870967741935
